In [3]:
import yelpapi
import json
import requests
from  pprint import  pprint
import pandas as pd
import numpy as np
import random

counties=pd.read_csv("acs2015_county_data.csv")
counties["county_string"] = counties["County"] + " County, " + counties["State"]
listofcounties = counties["county_string"].tolist()

In [4]:
zipcodes=pd.read_csv("zip_codes_states.csv")
zipcodes["county_string"] = zipcodes["county"] + " County, " + zipcodes["state"]
listofzipcodes=zipcodes["county_string"].unique().tolist()

In [5]:
#county list and zip list have discrepancy. need to pick counties that have entries in both lists

fullcountylist = list(set(listofcounties) & set(listofzipcodes))
sample_list = random.sample(fullcountylist,50)

In [10]:
#merged_df = zipcodes.merge(counties, left_on="county_string", right_on="county_string", how="left")
#maybe this wan't necessary but it's here if we need it
res_in_county = pd.DataFrame(sample_list)
res_in_county.columns = ["county_string"]
res_in_county["restaurant count"]=0
res_in_county["reviews"]=0
res_in_county["avg score"]=0
res_in_county["num of Italian"]=0
res_in_county["num of Mexican"]=0
res_in_county["num of Chinese"]=0
res_in_county["num of Indian"]=0
res_in_county.head()



,county_string,restaurant count,reviews,avg score,num of Italian,num of Mexican,num of Chinese,num of Indian
0,"Jasper County, Texas",0,0,0,0,0,0,0
1,"Itasca County, Minnesota",0,0,0,0,0,0,0
2,"Wayne County, New York",0,0,0,0,0,0,0
3,"Webster County, Nebraska",0,0,0,0,0,0,0
4,"Des Moines County, Iowa",0,0,0,0,0,0,0


In [11]:
res_df = pd.DataFrame(columns=["name","zipcode","county_string","rating","review_count","category"])
res_df

,name,zipcode,county_string,rating,review_count,category


In [8]:
r = 0
for c in sample_list:
    county_string = str(c)
    for term in ["Italian","Mexican","Chinese","Indian"]:
        #we are allowed 5000 queries per 24 hours. if we run out, we can create another. till then, use this
        yelpkey="IXRN_1GI8f8J9Kj5O5GciqeyFSWR5RMh41_rADGy1KrtQ5ZABGGZEkWwot5mOka0NlzlNxXPmzYXLWk88oOV0N_YxD1MSW4kQRfJnFrwfHLtKrr2HcsjLo6uWBzAWnYx"
        response = requests.get(url="https://api.yelp.com/v3/businesses/search", 
                                params= {"location": county_string, 
                                         "term": term, "limit":"50"}, 
                                headers={'Authorization': 'bearer %s' % yelpkey})
        responsejson = response.json()
        
        try:
            count=0
            
            for x in range(0,49):
                for y in range(0,len(responsejson["businesses"][x]["categories"])):
                    if ((responsejson["businesses"][x]["categories"][y]["title"]==term) and 
                        (responsejson["businesses"][x]["review_count"] > 0)):
                        
                        zc = int(responsejson["businesses"][x]["location"]["zip_code"])
                        if (zipcodes.loc[zipcodes["zip_code"]==zc]["county_string"].tolist()[0]== county_string):
                            
                            print("\n",responsejson["businesses"][x]["name"],
                                  responsejson["businesses"][x]["location"]["zip_code"],
                                  zipcodes.loc[zipcodes["zip_code"]==zc]["county_string"].tolist()[0],
                                  responsejson["businesses"][x]["rating"],
                                  responsejson["businesses"][x]["review_count"],
                                  responsejson["businesses"][x]["categories"][y]["title"])
                            
                            res_df.loc[r]=[responsejson["businesses"][x]["name"],
                                  zc,
                                  county_string,
                                  responsejson["businesses"][x]["rating"],
                                  responsejson["businesses"][x]["review_count"],
                                  term]
                            count += 1
                            r += 1
        except:
            print("")
    
        print("End of list. Found", count, term, "restaurants in", county_string)
  



 De Bella Sera Jasper 75951 Jasper County, Texas 4.0 33 Italian

 Pizza Hut 75951 Jasper County, Texas 1.5 6 Italian
End of list. Found 2 Italian restaurants in Jasper County, Texas

 MARTIN'S MEXICAN RESTAURANT 75951 Jasper County, Texas 4.0 17 Mexican

 Casa Olé 75951 Jasper County, Texas 3.0 13 Mexican

 Tia Juanita's Fish Camp 75951 Jasper County, Texas 4.5 6 Mexican
End of list. Found 3 Mexican restaurants in Jasper County, Texas

 Chinese Kitchen 75951 Jasper County, Texas 3.5 8 Chinese

 Super China Buffet 75951 Jasper County, Texas 1.5 6 Chinese

End of list. Found 2 Chinese restaurants in Jasper County, Texas

End of list. Found 0 Indian restaurants in Jasper County, Texas

End of list. Found 0 Italian restaurants in Itasca County, Minnesota

 Mucho Si 55769 Itasca County, Minnesota 4.0 12 Mexican

 El Potro Mexican Restaurant 55744 Itasca County, Minnesota 3.0 14 Mexican

 Effie Cafe 56639 Itasca County, Minnesota 4.0 5 Mexican

 Zorbaz 55744 Itasca County, Minnesota 3.5 22 


 El Tepeyac 08050 Ocean County, New Jersey 4.5 51 Mexican

 Tortilleria Rancho Grande 08050 Ocean County, New Jersey 4.5 60 Mexican

 Alberto's Deli 08731 Ocean County, New Jersey 4.5 9 Mexican

 El Familiar 08753 Ocean County, New Jersey 4.5 98 Mexican

 Los Aztecas Mexican Restaurant 08721 Ocean County, New Jersey 4.5 15 Mexican

 El Swell 08008 Ocean County, New Jersey 4.0 86 Mexican

 El Tenampa Mexican Restaurant 08050 Ocean County, New Jersey 3.5 46 Mexican

 La Bamba Mexican Restaurant 08008 Ocean County, New Jersey 4.0 63 Mexican

 Doña Julia Mexican Restaurant 08742 Ocean County, New Jersey 4.0 125 Mexican

 Mexico Lindo 08724 Ocean County, New Jersey 4.5 44 Mexican

 La Hacienda Tortilla factory and tacos 08757 Ocean County, New Jersey 4.5 11 Mexican

 Tornandez Restaurante Mexicano 08742 Ocean County, New Jersey 4.0 76 Mexican

 Tio Juan's Margaritas Mexican Restaurant 08753 Ocean County, New Jersey 3.0 80 Mexican

 Tenampa Mexican Restaurant 08755 Ocean County, New Jersey 


 Taco Bell 36804 Lee County, Alabama 5.0 2 Mexican

 Moe's Southwest Grill 36801 Lee County, Alabama 2.0 5 Mexican

 Durango Mexican Restaurant 36801 Lee County, Alabama 5.0 1 Mexican

 Chipotle Mexican Grill 36801 Lee County, Alabama 1.5 6 Mexican

 Chipotle Mexican Grill 36832 Lee County, Alabama 3.0 26 Mexican
End of list. Found 21 Mexican restaurants in Lee County, Alabama

 Mk's Asian Kitchen 36801 Lee County, Alabama 4.0 41 Chinese

 Wooden Chopsticks 36830 Lee County, Alabama 4.0 47 Chinese

 Taste of Asia 36801 Lee County, Alabama 3.5 25 Chinese

 Mandarin House Chinese 36801 Lee County, Alabama 3.5 34 Chinese

 Number 1 Asian Restaurant 36801 Lee County, Alabama 5.0 6 Chinese

 Wok 'n Roll Restaurant and Bar 36801 Lee County, Alabama 3.0 23 Chinese

 Hibachi Sushi Grill & Buffet 36801 Lee County, Alabama 3.0 25 Chinese

 Granny's Southern Cooking 36801 Lee County, Alabama 3.5 14 Chinese

 China Garden 36830 Lee County, Alabama 2.5 7 Chinese

 Panda 36832 Lee County, Alabama 3


End of list. Found 0 Indian restaurants in Anderson County, Tennessee

 Nostrano Italian Eatery 06468 Fairfield County, Connecticut 4.5 45 Italian

 Osteria Romana 06468 Fairfield County, Connecticut 4.5 62 Italian

 Trattoria 'A Vucchella 06604 Fairfield County, Connecticut 4.0 102 Italian

 Bartolo 06877 Fairfield County, Connecticut 4.0 66 Italian

 La Zingara 06801 Fairfield County, Connecticut 4.5 117 Italian

End of list. Found 5 Italian restaurants in Fairfield County, Connecticut

 Hacienda Don Manuel 06804 Fairfield County, Connecticut 4.0 71 Mexican

 Puerto Vallarta 06810 Fairfield County, Connecticut 3.5 81 Mexican

 Pancho's Tacos 06810 Fairfield County, Connecticut 4.0 144 Mexican

 El Coyote Restaurant 06468 Fairfield County, Connecticut 4.0 70 Mexican

End of list. Found 4 Mexican restaurants in Fairfield County, Connecticut

 Ren Dumpling & Noodle House 06897 Fairfield County, Connecticut 4.0 92 Chinese

 New Wok 06470 Fairfield County, Connecticut 4.0 28 Chinese

 Ka


 Olive Garden Italian Restaurant 27103 Forsyth County, North Carolina 3.0 41 Italian

 Francesco's 27105 Forsyth County, North Carolina 4.5 17 Italian

 Times Square Pizzeria 27127 Forsyth County, North Carolina 4.0 56 Italian

 Piccolo Milano Pizza & Restaurant 27051 Forsyth County, North Carolina 3.5 14 Italian

 Whitaker Square Pizza 27106 Forsyth County, North Carolina 4.5 17 Italian

 Silvia's Honey Tree Restaurant 27105 Forsyth County, North Carolina 4.0 36 Italian

 Sixty Six Pizzeria Bar & Grill 27103 Forsyth County, North Carolina 3.5 71 Italian

 Illianos pizza 27012 Forsyth County, North Carolina 4.0 17 Italian
End of list. Found 33 Italian restaurants in Forsyth County, North Carolina

 El Rancho Taqueria 27107 Forsyth County, North Carolina 4.5 131 Mexican

 La Casa Lopez Mexican Grill 27103 Forsyth County, North Carolina 4.5 61 Mexican

 Antojitos Las Delicias 27101 Forsyth County, North Carolina 5.0 25 Mexican

 El Maguey Restaurant 27040 Forsyth County, North Carolina 


 Szechuan Chinese Restaurant 74006 Washington County, Oklahoma 4.0 20 Chinese

 Ocean China 74006 Washington County, Oklahoma 4.0 17 Chinese

 Asian Cafe 74006 Washington County, Oklahoma 4.0 9 Chinese

 Panda Express 74006 Washington County, Oklahoma 3.0 2 Chinese

 New China 74006 Washington County, Oklahoma 2.5 7 Chinese

 Hunan Chinese Restaurant 74006 Washington County, Oklahoma 2.0 14 Chinese
End of list. Found 6 Chinese restaurants in Washington County, Oklahoma

End of list. Found 0 Indian restaurants in Washington County, Oklahoma

 Pizza Hut 47371 Jay County, Indiana 4.0 2 Italian
End of list. Found 1 Italian restaurants in Jay County, Indiana

 El Camino Real Restaurant 47371 Jay County, Indiana 3.5 12 Mexican

 Taco Bell 47371 Jay County, Indiana 3.0 1 Mexican
End of list. Found 2 Mexican restaurants in Jay County, Indiana

 Great Wall Chinese Restaurant 47371 Jay County, Indiana 2.5 3 Chinese
End of list. Found 1 Chinese restaurants in Jay County, Indiana

End of list. Fo


 Piero's PizzaVino 92260 Riverside County, California 4.0 581 Italian

 Birba 92262 Riverside County, California 4.0 707 Italian

 La Spiga Ristorante Italiano 92260 Riverside County, California 4.5 169 Italian

 Bucatini Trattoria 92253 Riverside County, California 3.0 241 Italian

 Trilussa Ristorante 92234 Riverside County, California 4.0 212 Italian

 Cafe Italia 92210 Riverside County, California 4.0 147 Italian

 Bucatini Trattoria 92211 Riverside County, California 3.5 77 Italian

 Tiramisu Trattoria 92260 Riverside County, California 4.0 65 Italian

 Il Corso 92260 Riverside County, California 4.0 213 Italian

 Bontá Restaurant & Bar 92234 Riverside County, California 4.5 207 Italian

 Francesco's Cafe 92270 Riverside County, California 4.0 151 Italian

 Kalura Trattoria 92262 Riverside County, California 3.5 331 Italian

 Le Donne Cucina Italiana 92260 Riverside County, California 3.5 78 Italian

 Giuseppe's Pizza & Pasta 92264 Riverside County, California 4.0 491 Italian

 B


 Tres Locos 13428 Montgomery County, New York 3.0 7 Mexican

 Moe's Southwest Grill 12010 Montgomery County, New York 4.0 10 Mexican

 Taco Bell 12010 Montgomery County, New York 4.0 2 Mexican
End of list. Found 3 Mexican restaurants in Montgomery County, New York

 New Peking In Fonda 12068 Montgomery County, New York 4.5 8 Chinese

 New China 13428 Montgomery County, New York 3.5 4 Chinese

 RED LANTERN HOUSE 13339 Montgomery County, New York 4.5 2 Chinese

 China Wok Chinese Restaurant 12010 Montgomery County, New York 3.5 4 Chinese

 China One 12010 Montgomery County, New York 2.0 5 Chinese
End of list. Found 5 Chinese restaurants in Montgomery County, New York

End of list. Found 0 Indian restaurants in Montgomery County, New York

 Dominic's Italian Restaurant 44256 Medina County, Ohio 3.5 45 Italian

 Samosky's Homestyle Pizzeria 44280 Medina County, Ohio 4.0 82 Italian

 Digsby's Pizza Pasta & Pub Grub 44273 Medina County, Ohio 4.5 15 Italian

 Romeo's Italian Kitchen 44256 Me

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
res_df

,name,zipcode,county_string,rating,review_count,category
0,De Bella Sera Jasper,75951,"Jasper County, Texas",4.0,33,Italian
1,Pizza Hut,75951,"Jasper County, Texas",1.5,6,Italian
2,MARTIN'S MEXICAN RESTAURANT,75951,"Jasper County, Texas",4.0,17,Mexican
3,Casa Olé,75951,"Jasper County, Texas",3.0,13,Mexican
4,Tia Juanita's Fish Camp,75951,"Jasper County, Texas",4.5,6,Mexican
5,Chinese Kitchen,75951,"Jasper County, Texas",3.5,8,Chinese
6,Super China Buffet,75951,"Jasper County, Texas",1.5,6,Chinese
7,Mucho Si,55769,"Itasca County, Minnesota",4.0,12,Mexican
8,El Potro Mexican Restaurant,55744,"Itasca County, Minnesota",3.0,14,Mexican
9,Effie Cafe,56639,"Itasca County, Minnesota",4.0,5,Mexican


In [ ]:
responsejson["businesses"]